In [1]:
import os
import toml
import pandas as pd
from utils.config_helper import update_nested_toml, load_config

breast
lung
prostate
stomach
rectal

In [2]:
TYPE = input("Enter the type of the config file: ")
CONFIG_PATH = f"../config/{TYPE}.toml"
config = load_config(CONFIG_PATH)

In [3]:
def inspect_nan(df, name):
    print(df[pd.isna(df[name])])

In [4]:
beta_file_number = config["init"]["hyper"]["beta_file_number"]
test_ratio = config["init"]["hyper"]["test_ratio"]
seed = config["init"]["hyper"]["splitting_seed"]
normal_number_0 = config["init"]["hyper"]["normal_number_0"]
if beta_file_number == 2:
    normal_number_1 = config["init"]["hyper"]["normal_number_1"]
data_source = config["init"]["hyper"]["data_source"]
is_columns_duplicated = config["init"]["hyper"]["is_columns_duplicated"]
if data_source == "GDC_stomach_GSE99553":  # god forgive me
    is_columns_duplicated_1 = config["init"]["hyper"]["is_columns_duplicated_1"]
is_oversample = config["init"]["hyper"]["is_oversample"]

In [5]:
if is_oversample:
    print("train on oversampled dataset.")
    trainOutPath = f"../{TYPE}/result/{data_source}/train{int(100-test_ratio*100)}_oversample"
else:
    print("train on original dataset.")
    trainOutPath = f"../{TYPE}/result/{data_source}/train{int(100-test_ratio*100)}"
testOutPath = f"../{TYPE}/result/{data_source}/test{int(test_ratio*100)}"

train on original dataset.


### Section. 0 Merge and Split Champ Data
- setting is_oversample = 0 to make sure three datasets are stored properly

- file paths
  - {TYPE}/result/{data_source}/test20/all_beta_normalized_1.csv
  - {TYPE}/result/{data_source}/train80/all_beta_normalized_0.csv
  - {TYPE}/result/{data_source}/train80_oversample/all_beta_normalized_0_oversample.csv


#### 0.1 Merge Dataset (if possible)

In [41]:
df0 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_0.csv")

In [42]:
if beta_file_number == 2:
    df1 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_1.csv")

In [43]:
# DEBUG
df0
# END

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,909,910,911,912,913,914,915,916,917,918
0,cg00000957,0.870131,0.870131,0.855272,0.855272,0.837964,0.837964,0.881787,0.881787,0.840490,...,0.847760,0.847760,0.828761,0.828761,0.854923,0.854923,0.845223,0.845223,0.868674,0.868674
1,cg00001349,0.805265,0.805265,0.786301,0.786301,0.812133,0.812133,0.702542,0.702542,0.794556,...,0.827351,0.827351,0.795160,0.795160,0.783938,0.783938,0.740427,0.740427,0.517052,0.517052
2,cg00001583,0.130379,0.130379,0.105195,0.105195,0.069474,0.069474,0.065219,0.065219,0.083583,...,0.241279,0.241279,0.413970,0.413970,0.272795,0.272795,0.443902,0.443902,0.051824,0.051824
3,cg00002028,0.027859,0.027859,0.028367,0.028367,0.035275,0.035275,0.009818,0.009818,0.040193,...,0.031782,0.031782,0.030371,0.030371,0.043669,0.043669,0.052163,0.052163,0.032497,0.032497
4,cg00002719,0.024905,0.024905,0.028116,0.028116,0.026357,0.026357,0.031588,0.031588,0.049386,...,0.523305,0.523305,0.032755,0.032755,0.463249,0.463249,0.254924,0.254924,0.715903,0.715903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120672,cg15728744,0.989299,0.989299,0.984082,0.984082,0.975333,0.975333,0.992034,0.992034,0.959184,...,0.970321,0.970321,0.885888,0.885888,0.892856,0.892856,0.989962,0.989962,0.959804,0.959804
120673,cg15728779,0.455539,0.455539,0.461929,0.461929,0.397068,0.397068,0.394965,0.394965,0.405328,...,0.350407,0.350407,0.295748,0.295748,0.320699,0.320699,0.223251,0.223251,0.235275,0.235275
120674,cg15729771,0.057365,0.057365,0.049908,0.049908,0.061390,0.061390,0.045488,0.045488,0.048277,...,0.065144,0.065144,0.072834,0.072834,0.053064,0.053064,0.046125,0.046125,0.077101,0.077101
120675,cg15730248,0.958313,0.958313,0.954312,0.954312,0.953241,0.953241,0.962748,0.962748,0.928654,...,0.843446,0.843446,0.926867,0.926867,0.895288,0.895288,0.968495,0.968495,0.951945,0.951945


In [44]:
# DEBUG
df1
# END

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,909,910,911,912,913,914,915,916,917,918
0,cg00000957,0.873237,0.873237,0.870941,0.870941,0.859828,0.859828,0.883027,0.883027,0.878071,...,0.799261,0.799261,0.330985,0.330985,0.860865,0.860865,0.869139,0.869139,0.865573,0.865573
1,cg00001349,0.813589,0.813589,0.779412,0.779412,0.881907,0.881907,0.740944,0.740944,0.848596,...,0.835029,0.835029,0.780882,0.780882,0.716940,0.716940,0.752071,0.752071,0.859589,0.859589
2,cg00001583,0.060897,0.060897,0.071685,0.071685,0.073219,0.073219,0.079168,0.079168,0.055963,...,0.547632,0.547632,0.658158,0.658158,0.196573,0.196573,0.238353,0.238353,0.687156,0.687156
3,cg00002028,0.020690,0.020690,0.025741,0.025741,0.011456,0.011456,0.014352,0.014352,0.032681,...,0.031653,0.031653,0.056207,0.056207,0.044863,0.044863,0.038549,0.038549,0.023179,0.023179
4,cg00002719,0.031898,0.031898,0.023960,0.023960,0.047802,0.047802,0.025617,0.025617,0.039806,...,0.657312,0.657312,0.687780,0.687780,0.063309,0.063309,0.076774,0.076774,0.673089,0.673089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360237,cg27656573,0.958146,0.958146,0.956567,0.956567,0.960663,0.960663,0.952006,0.952006,0.961032,...,0.941653,0.941653,0.878673,0.878673,0.955497,0.955497,0.947089,0.947089,0.963040,0.963040
360238,cg27657363,0.949939,0.949939,0.955772,0.955772,0.945914,0.945914,0.958400,0.958400,0.967471,...,0.660955,0.660955,NaN,NaN,0.925782,0.925782,0.925081,0.925081,0.804342,0.804342
360239,cg27657537,0.069554,0.069554,0.055817,0.055817,0.045695,0.045695,0.049810,0.049810,0.053926,...,0.032993,0.032993,NaN,NaN,0.088574,0.088574,0.111038,0.111038,0.087881,0.087881
360240,cg27662611,0.027544,0.027544,0.027796,0.027796,0.044085,0.044085,0.022501,0.022501,0.058010,...,0.038424,0.038424,NaN,NaN,0.029692,0.029692,0.048457,0.048457,0.053156,0.053156


In [45]:
# potential feature loss
if beta_file_number == 2:
    feature_name_0 = df0.iloc[:, 0].tolist()
    feature_name_1 = df1.iloc[:, 0].tolist()

    feature_name = list(set(feature_name_0).intersection(feature_name_1))
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name_0)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_1", len(feature_name_1)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_intersection", len(feature_name)
    )
elif beta_file_number == 1:
    feature_name = df0.iloc[:, 0].tolist()
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name)
    )

In [46]:
if beta_file_number == 2:
    df0_join = df0[df0.iloc[:, 0].isin(feature_name)]
    df1_join = df1[df1.iloc[:, 0].isin(feature_name)]

In [47]:
if beta_file_number == 2:
    df0_join = df0_join.iloc[:, 1::is_columns_duplicated]
    if data_source == "GDC_stomach_GSE99553":  # god forgive me
        df1_join = df1_join.iloc[:, 1::is_columns_duplicated_1]
    else:
        df1_join = df1_join.iloc[:, 1::is_columns_duplicated]
    df0_join.reset_index(drop=True, inplace=True)
    df1_join.reset_index(drop=True, inplace=True)
    df0_join_normal = df0_join.iloc[:, :normal_number_0]
    df0_join_tumor = df0_join.iloc[:, normal_number_0:]
    df1_join_normal = df1_join.iloc[:, :normal_number_1]
    df1_join_tumor = df1_join.iloc[:, normal_number_1:]
elif beta_file_number == 1:
    df0_join = df0.iloc[:, 1::is_columns_duplicated]

In [48]:
if beta_file_number == 2:
    df_normal = pd.concat([df0_join_normal, df1_join_normal], axis=1)
    df_tumor = pd.concat([df0_join_tumor, df1_join_tumor], axis=1)

In [49]:
# drop those samples with missing value
# note: could use padding or other methods to fill the missing value

if beta_file_number == 2:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfc_shape", df_tumor.shape
    )
    df_normal.dropna(inplace=True, axis=1)
    df_tumor.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfc_shape", df_tumor.shape
    )
elif beta_file_number == 1:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_df_shape", df0_join.shape
    )
    df0_join.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_df_shape", df0_join.shape
    )

In [50]:
# Combine the normal and tumor data
if beta_file_number == 2:
    X = pd.concat([df_normal, df_tumor], axis=1).T
    y = [0] * df_normal.shape[1] + [1] * df_tumor.shape[1]
elif beta_file_number == 1:
    X = df0_join.T
    y = [0] * normal_number_0 + [1] * (df0_join.shape[1] - normal_number_0)


#### 0.2 Split Dataset

In [51]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_ratio, random_state=seed
)

In [52]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,115109,115110,115111,115112,115113,115114,115115,115116,115117,115118
189,0.841756,0.703298,0.431084,0.027505,0.397063,0.465172,0.249965,0.949241,0.012780,0.262894,...,0.027010,0.920087,0.850160,0.930655,0.791388,0.989132,0.384792,0.038485,0.973749,0.679401
15,0.878719,0.857571,0.079447,0.032354,0.046182,0.241271,0.315196,0.968882,0.026012,0.132082,...,0.022753,0.970901,0.661749,0.928384,0.682931,0.984485,0.495349,0.067969,0.942289,0.958157
67,0.802070,0.797716,0.096851,0.027988,0.017034,0.310766,0.233277,0.955857,0.015123,0.117587,...,0.040264,0.977996,0.726119,0.917069,0.759037,0.984149,0.420042,0.049243,0.947436,0.960082
761,0.862710,0.833682,0.316148,0.049158,0.329400,0.414738,0.204136,0.967899,0.014163,0.181442,...,0.024757,0.966543,0.646962,0.801688,0.803312,0.990239,0.416372,0.066303,0.980574,0.963004
343,0.892635,0.871973,0.510363,0.017838,0.430559,0.181806,0.155549,0.973625,0.022387,0.045736,...,0.029056,0.739023,0.643690,0.951082,0.654773,0.977833,0.497420,0.067781,0.979291,0.956772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.903994,0.701379,0.092457,0.050638,0.013913,0.236373,0.254050,0.954160,0.014197,0.082760,...,0.085104,0.970894,0.679033,0.891433,0.687496,0.989801,0.278334,0.027561,0.908630,0.883741
139,0.848019,0.714955,0.345282,0.044060,0.217289,0.286300,0.213746,0.933303,0.019338,0.163362,...,0.040914,0.936746,0.703963,0.932591,0.794732,0.973849,0.443481,0.042136,0.945318,0.949573
467,0.842057,0.844408,0.167503,0.030814,0.045414,0.271550,0.299056,0.955565,0.011885,0.289069,...,0.015548,0.949698,0.737791,0.930943,0.696165,0.968152,0.431557,0.055152,0.935072,0.805257
285,0.875423,0.607010,0.052774,0.022742,0.015006,0.178868,0.350928,0.967761,0.011369,0.070731,...,0.038441,0.555965,0.206392,0.409093,0.777240,0.976052,0.203283,0.056329,0.972555,0.937175


In [53]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,115109,115110,115111,115112,115113,115114,115115,115116,115117,115118
361,0.854024,0.756719,0.316806,0.014673,0.123875,0.450886,0.256985,0.920395,0.016650,0.283159,...,0.022427,0.957761,0.736881,0.943776,0.821242,0.982496,0.363262,0.044705,0.957992,0.933558
129,0.823536,0.693220,0.111985,0.037636,0.064150,0.382674,0.285728,0.932670,0.038226,0.249276,...,0.025023,0.975357,0.831464,0.921149,0.713270,0.973149,0.347267,0.053407,0.935170,0.937511
193,0.862103,0.829389,0.068538,0.034424,0.184173,0.353739,0.273153,0.944550,0.030437,0.300550,...,0.029220,0.922292,0.703057,0.921094,0.727430,0.977570,0.470799,0.060327,0.951799,0.949233
427,0.873949,0.844888,0.032152,0.025734,0.031938,0.490008,0.136191,0.957389,0.023358,0.105509,...,0.051755,0.965039,0.882884,0.913865,0.901746,0.992619,0.287230,0.079063,0.977999,0.966018
525,0.880170,0.825625,0.311102,0.056664,0.165325,0.378298,0.257684,0.951765,0.022979,0.241494,...,0.032187,0.905780,0.618101,0.880485,0.776595,0.958387,0.592535,0.065785,0.902132,0.944908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.841512,0.734382,0.313935,0.047603,0.035616,0.141369,0.117212,0.943620,0.025846,0.072433,...,0.035184,0.837315,0.613648,0.687551,0.518779,0.899735,0.294387,0.145664,0.887406,0.537128
203,0.852130,0.851036,0.394213,0.043161,0.348362,0.385916,0.181517,0.956662,0.022443,0.147065,...,0.038196,0.897077,0.658867,0.915679,0.700238,0.985145,0.479830,0.066491,0.954273,0.957002
837,0.868395,0.724567,0.351308,0.036196,0.158584,0.431912,0.306839,0.942866,0.031599,0.164135,...,0.018459,0.949499,0.809953,0.938143,0.786967,0.979831,0.452258,0.066641,0.951093,0.969676
65,0.876630,0.811409,0.093006,0.017022,0.026142,0.252944,0.305223,0.971243,0.017526,0.080169,...,0.043310,0.979285,0.755543,0.914282,0.717756,0.985018,0.484947,0.033403,0.961524,0.965480


#### 0.3 Oversample

In [54]:
update_nested_toml(
    "preprocess.merge_and_split", "Before_SMOTE_X_train_shape", X_train.shape
)
update_nested_toml(
    "preprocess.merge_and_split", "Before_SMOTE_y_train_shape", len(y_train)
)

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=seed)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
update_nested_toml(
    "preprocess.merge_and_split", "After_SMOTE_X_train_shape", X_resampled.shape
)
update_nested_toml(
    "preprocess.merge_and_split", "After_SMOTE_y_train_shape", len(y_resampled)
)


In [55]:
X_resampled

,0,1,2,3,4,5,6,7,8,9,...,115109,115110,115111,115112,115113,115114,115115,115116,115117,115118
0,0.841756,0.703298,0.431084,0.027505,0.397063,0.465172,0.249965,0.949241,0.012780,0.262894,...,0.027010,0.920087,0.850160,0.930655,0.791388,0.989132,0.384792,0.038485,0.973749,0.679401
1,0.878719,0.857571,0.079447,0.032354,0.046182,0.241271,0.315196,0.968882,0.026012,0.132082,...,0.022753,0.970901,0.661749,0.928384,0.682931,0.984485,0.495349,0.067969,0.942289,0.958157
2,0.802070,0.797716,0.096851,0.027988,0.017034,0.310766,0.233277,0.955857,0.015123,0.117587,...,0.040264,0.977996,0.726119,0.917069,0.759037,0.984149,0.420042,0.049243,0.947436,0.960082
3,0.862710,0.833682,0.316148,0.049158,0.329400,0.414738,0.204136,0.967899,0.014163,0.181442,...,0.024757,0.966543,0.646962,0.801688,0.803312,0.990239,0.416372,0.066303,0.980574,0.963004
4,0.892635,0.871973,0.510363,0.017838,0.430559,0.181806,0.155549,0.973625,0.022387,0.045736,...,0.029056,0.739023,0.643690,0.951082,0.654773,0.977833,0.497420,0.067781,0.979291,0.956772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0.878729,0.865951,0.089815,0.021578,0.023333,0.289873,0.295743,0.967093,0.015089,0.120200,...,0.036278,0.982500,0.759131,0.931467,0.737107,0.984409,0.463074,0.061237,0.962958,0.972720
1082,0.814309,0.765022,0.093404,0.026295,0.019902,0.322114,0.232296,0.955015,0.012538,0.112805,...,0.039917,0.975547,0.716257,0.917594,0.744179,0.983871,0.435207,0.047847,0.939098,0.957803
1083,0.849890,0.897346,0.067199,0.011073,0.031814,0.308065,0.240790,0.975927,0.016051,0.064191,...,0.044021,0.975818,0.638132,0.951337,0.741305,0.987540,0.402084,0.056576,0.958052,0.967637
1084,0.864610,0.707271,0.100759,0.023600,0.029610,0.336432,0.252440,0.954921,0.022920,0.120200,...,0.033460,0.973117,0.647463,0.915533,0.692098,0.981847,0.324761,0.047862,0.916655,0.956687


In [59]:
train_class_distribution_oversample = Counter(y_resampled)
train_class_distribution = Counter(y_train)
testing_class_distribution = Counter(y_test)

update_nested_toml("preprocess.merge_and_split", "training_set_samples_oversample", len(X_resampled))
update_nested_toml("preprocess.merge_and_split", "training_set_samples", len(X_train))
update_nested_toml("preprocess.merge_and_split", "testing_set_samples", len(X_test))
update_nested_toml(
    "preprocess.merge_and_split", 
    "training_class_distribution_oversample", 
    [train_class_distribution_oversample[0], train_class_distribution_oversample[1]]
)
update_nested_toml(
    "preprocess.merge_and_split",
    "training_class_distribution",
    [train_class_distribution[0], train_class_distribution[1]],
)
update_nested_toml(
    "preprocess.merge_and_split",
    "testing_class_distribution",
    [testing_class_distribution[0], testing_class_distribution[1]],
)

In [60]:
X_resampled.columns = feature_name
X_resampled["label"] = y_resampled
X_resampled.sort_values(by=["label"], inplace=True)
train_df_os = X_resampled.T
train_df_os.columns = range(train_df_os.shape[1])

X_train.columns = feature_name
X_train["label"] = y_train
X_train.sort_values(by=["label"], inplace=True)
train_df = X_train.T
train_df.columns = range(train_df.shape[1])

X_test.columns = feature_name
X_test["label"] = y_test
X_test.sort_values(by=["label"], inplace=True)
test_df = X_test.T
test_df.columns = range(test_df.shape[1])

train_df_os.insert(0, "Unnamed: 0", train_df_os.index)
train_df_os.reset_index(drop=True, inplace=True)
train_df.insert(0, "Unnamed: 0", train_df.index)
train_df.reset_index(drop=True, inplace=True)
test_df.insert(0, "Unnamed: 0", test_df.index)
test_df.reset_index(drop=True, inplace=True)

C:\Users\Vincent\AppData\Local\Temp\ipykernel_2208\3479401642.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [61]:
train_df_os

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085
0,cg07301377,0.852370,0.870567,0.891173,0.878071,0.867501,0.880786,0.818329,0.882626,0.888366,...,0.863748,0.880139,0.832182,0.850228,0.866454,0.824812,0.865056,0.892635,0.862710,0.841756
1,cg14101485,0.771133,0.811704,0.857874,0.848596,0.682805,0.728048,0.665811,0.827413,0.720627,...,0.774240,0.815669,0.632477,0.800390,0.742772,0.758846,0.749751,0.871973,0.833682,0.703298
2,cg15648026,0.143240,0.077060,0.061272,0.055963,0.099386,0.126752,0.090545,0.086449,0.076482,...,0.417835,0.697842,0.516217,0.568682,0.477536,0.179259,0.049682,0.510363,0.316148,0.431084
3,cg11078358,0.035310,0.057602,0.012510,0.032681,0.022125,0.035585,0.031436,0.017390,0.035903,...,0.059408,0.043402,0.032966,0.049887,0.033244,0.020079,0.023211,0.017838,0.049158,0.027505
4,cg26243254,0.034194,0.049185,0.031553,0.039806,0.030072,0.029031,0.056112,0.022634,0.026980,...,0.542697,0.613503,0.148921,0.449114,0.492683,0.283667,0.103404,0.430559,0.329400,0.397063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115115,cg04035353,0.363306,0.500272,0.518306,0.483960,0.282297,0.385297,0.424869,0.421890,0.416448,...,0.229761,0.192158,0.373584,0.508599,0.385173,0.395531,0.251555,0.497420,0.416372,0.384792
115116,cg09428340,0.062891,0.096943,0.031127,0.055751,0.047229,0.036478,0.073008,0.045607,0.029354,...,0.074228,0.067546,0.055559,0.062601,0.044367,0.041907,0.040397,0.067781,0.066303,0.038485
115117,cg07434447,0.946201,0.890536,0.962135,0.942869,0.904997,0.939219,0.908405,0.965903,0.935189,...,0.881388,0.981038,0.934954,0.908864,0.963559,0.938969,0.905560,0.979291,0.980574,0.973749
115118,cg11152364,0.945514,0.949398,0.940530,0.971369,0.954035,0.931497,0.922656,0.962282,0.964943,...,0.954805,0.936784,0.912121,0.929484,0.774787,0.902551,0.809405,0.956772,0.963004,0.679401


In [62]:
# DEBUG
train_df
# END

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,590,591,592,593,594,595,596,597,598,599
0,cg07301377,0.903994,0.878719,0.802070,0.879334,0.880786,0.818329,0.882195,0.858834,0.851540,...,0.851061,0.828820,0.892635,0.877953,0.862710,0.848019,0.842057,0.875423,0.843760,0.841756
1,cg14101485,0.701379,0.857571,0.797716,0.682503,0.728048,0.665811,0.796505,0.893290,0.874451,...,0.727444,0.548354,0.871973,0.858174,0.833682,0.714955,0.844408,0.607010,0.799665,0.703298
2,cg15648026,0.092457,0.079447,0.096851,0.087588,0.126752,0.090545,0.077337,0.061910,0.088524,...,0.131189,0.357203,0.510363,0.149980,0.316148,0.345282,0.167503,0.052774,0.430911,0.431084
3,cg11078358,0.050638,0.032354,0.027988,0.027399,0.035585,0.031436,0.026610,0.024558,0.032837,...,0.027541,0.005888,0.017838,0.012281,0.049158,0.044060,0.030814,0.022742,0.028418,0.027505
4,cg26243254,0.013913,0.046182,0.017034,0.017774,0.029031,0.056112,0.044048,0.046654,0.071240,...,0.050009,0.047248,0.430559,0.053022,0.329400,0.217289,0.045414,0.015006,0.322526,0.397063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115115,cg04035353,0.278334,0.495349,0.420042,0.418839,0.385297,0.424869,0.305107,0.334699,0.464884,...,0.547382,0.350633,0.497420,0.418386,0.416372,0.443481,0.431557,0.203283,0.522833,0.384792
115116,cg09428340,0.027561,0.067969,0.049243,0.049530,0.036478,0.073008,0.040758,0.061940,0.065148,...,0.061957,0.042510,0.067781,0.058195,0.066303,0.042136,0.055152,0.056329,0.066074,0.038485
115117,cg07434447,0.908630,0.942289,0.947436,0.929303,0.939219,0.908405,0.951541,0.932398,0.941606,...,0.947187,0.837662,0.979291,0.962619,0.980574,0.945318,0.935072,0.972555,0.957475,0.973749
115118,cg11152364,0.883741,0.958157,0.960082,0.951386,0.931497,0.922656,0.939522,0.944974,0.966774,...,0.969555,0.497398,0.956772,0.959420,0.963004,0.949573,0.805257,0.937175,0.943723,0.679401


In [63]:
# DEBUG
test_df
# END

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,141,142,143,144,145,146,147,148,149,150
0,cg07301377,0.874302,0.859828,0.883532,0.882055,0.844243,0.858966,0.841300,0.896747,0.837964,...,0.869053,0.835085,0.853119,0.844752,0.867991,0.841512,0.856398,0.852130,0.868395,0.863374
1,cg14101485,0.786932,0.881907,0.847782,0.848587,0.793554,0.775272,0.810149,0.860436,0.812133,...,0.672140,0.850578,0.444381,0.770161,0.750688,0.734382,0.536156,0.851036,0.724567,0.871801
2,cg15648026,0.102171,0.073219,0.108394,0.123222,0.099305,0.055171,0.139665,0.083554,0.069474,...,0.139428,0.603300,0.032842,0.286449,0.425047,0.313935,0.499903,0.394213,0.351308,0.651526
3,cg11078358,0.025665,0.011456,0.029587,0.085896,0.028458,0.025899,0.037926,0.010474,0.035275,...,0.048695,0.052466,0.029943,0.055550,0.025035,0.047603,0.037963,0.043161,0.036196,0.053677
4,cg26243254,0.026789,0.047802,0.044525,0.063606,0.025645,0.019617,0.062630,0.010090,0.026357,...,0.049087,0.707763,0.763141,0.347708,0.359921,0.035616,0.732572,0.348362,0.158584,0.760026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115115,cg04035353,0.441116,0.438248,0.430714,0.380426,0.440188,0.251832,0.536738,0.392736,0.397068,...,0.532671,0.165531,0.452501,0.296396,0.317538,0.294387,0.229584,0.479830,0.452258,0.793403
115116,cg09428340,0.056167,0.039365,0.053691,0.077322,0.071702,0.045686,0.045855,0.048512,0.061390,...,0.066911,0.057710,0.067335,0.060673,0.029639,0.145664,0.063733,0.066491,0.066641,0.071990
115117,cg07434447,0.972769,0.962961,0.937107,0.946231,0.943473,0.955075,0.953047,0.961717,0.953241,...,0.954063,0.971745,0.952562,0.857297,0.948850,0.887406,0.514595,0.954273,0.951093,0.962310
115118,cg11152364,0.957371,0.949991,0.947887,0.955418,0.969292,0.953709,0.916590,0.976610,0.950154,...,0.952465,0.777661,0.950322,0.804706,0.960664,0.537128,0.711278,0.957002,0.969676,0.957601


In [64]:
os.makedirs(f"{trainOutPath}", exist_ok=True)
os.makedirs(
    f"{testOutPath}",
    exist_ok=True,
)
train_df_os.to_csv(f"{trainOutPath}_oversample/all_beta_normalized_0_oversample.csv", index=False)
train_df.to_csv(f"{trainOutPath}/all_beta_normalized_0.csv", index=False)
test_df.to_csv(f"{testOutPath}/all_beta_normalized_1.csv", index=False)

In [65]:
del train_df, test_df, train_df_os

#### 0.4 Upload Datasets

In [66]:
zip_filename = f"{TYPE}_beta_files.zip"

In [67]:
import zipfile

with zipfile.ZipFile(zip_filename, "w") as zipf:
    zipf.write(
        f"{trainOutPath}/all_beta_normalized_0_oversample.csv",
        arcname="all_beta_normalized_0_oversample.csv",
    )
    zipf.write(
        f"{trainOutPath}/all_beta_normalized_0.csv", arcname="all_beta_normalized_0.csv"
    )
    zipf.write(
        f"{testOutPath}/all_beta_normalized_1.csv", arcname="all_beta_normalized_1.csv"
    )

In [43]:
from api import utils
service = utils.authenticate_drive()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=260407430533-mtnvh98d2pv3eo7mgb14h6en5827k7b5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A14728%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=zkRitlI3rK9xVLPC7hIwofYz2hRV69&access_type=offline


INFO:google_auth_oauthlib.flow:"GET /?state=zkRitlI3rK9xVLPC7hIwofYz2hRV69&code=4/0AanRRrsAlCFZqdvSdq6thRCYlOzYqD9I8K-a_1YLt2RqCmkF8VMAsxo8pBEnxU8rhn6-WQ&scope=https://www.googleapis.com/auth/drive.file HTTP/1.1" 200 65
INFO:api.utils:New credentials obtained.
INFO:api.utils:Credentials saved to token.pickle.
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


In [44]:
directory = utils.create_folder(service, TYPE)

Created folder 'lung' with ID: 1jyrsvxFbTjgrf4Pjf9S5eKfgOVycNQGo


In [45]:
utils.run_upload_with_separate_thread(service, directory, zip_filename)

INFO:api.utils:Upload 0% complete for lung_beta_files.zip
INFO:api.utils:Upload 0% complete for lung_beta_files.zip
INFO:api.utils:Upload 0% complete for lung_beta_files.zip
INFO:api.utils:Upload 0% complete for lung_beta_files.zip
INFO:api.utils:Upload 0% complete for lung_beta_files.zip
INFO:api.utils:Upload 1% complete for lung_beta_files.zip
INFO:api.utils:Upload 1% complete for lung_beta_files.zip
INFO:api.utils:Upload 1% complete for lung_beta_files.zip
INFO:api.utils:Upload 1% complete for lung_beta_files.zip
INFO:api.utils:Upload 1% complete for lung_beta_files.zip
INFO:api.utils:Upload 2% complete for lung_beta_files.zip
INFO:api.utils:Upload 2% complete for lung_beta_files.zip
INFO:api.utils:Upload 2% complete for lung_beta_files.zip
INFO:api.utils:Upload 2% complete for lung_beta_files.zip
INFO:api.utils:Upload 2% complete for lung_beta_files.zip
INFO:api.utils:Upload 3% complete for lung_beta_files.zip
INFO:api.utils:Upload 3% complete for lung_beta_files.zip
INFO:api.utils

### Sec. 1 Delta Beta Calculation

In [11]:
import gdown

#### 1.1 Download

In [ ]:
url = input()

output = 'download.zip'

In [ ]:
gdown.download(url, output, quiet=False)

In [13]:
import zipfile
import shutil

with zipfile.ZipFile("download.zip", "r") as zip_ref:
    zip_ref.extractall("download")

shutil.move("download/all_beta_normalized_0.csv", f"all_beta_normalized_0.csv")
shutil.move("download/all_beta_normalized_1.csv", f"all_beta_normalized_1.csv")

os.remove("download.zip")
shutil.rmtree("download")

#### 1.2 Average Delta Beta Calculation

In [6]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [7]:
# remove outlier in terms of every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR * 1.5
    lower_fence = Q1 - IQR * 1.5
    return upper_fence, lower_fence


def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf = df[(df > lower_fence) & (df < upper_fence)]
    return ddf

In [8]:
config = load_config(CONFIG_PATH)
normal_count = config["preprocess"]["merge_and_split"]["training_class_distribution"][0]
all_beta_normalized_normal = train_df.iloc[:-1, 1 : normal_count + 1 :].T


all_beta_normalized_tumor = train_df.iloc[:-1, normal_count + 1 : :].T

In [9]:
all_beta_normalized_normal = no_outlier(all_beta_normalized_normal)
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [10]:
train_normal_avg = all_beta_normalized_normal.mean(skipna=True, axis=0)

In [11]:
train_normal_avg

0         0.866381
1         0.787502
2         0.089028
3         0.028078
4         0.032265
            ...   
115114    0.984131
115115    0.415486
115116    0.049014
115117    0.947244
115118    0.956326
Length: 115119, dtype: float64

In [12]:
all_beta_normalized_tumor

,0,1,2,3,4,5,6,7,8,9,...,115109,115110,115111,115112,115113,115114,115115,115116,115117,115118
57,0.857447,0.803771,0.074472,0.034362,0.257102,0.159819,0.111679,0.950666,0.009761,0.124913,...,0.066841,0.581180,0.555451,0.897070,0.756273,0.982374,0.334648,0.058743,0.942200,0.965174
58,0.871969,0.613322,0.339645,0.044314,0.183437,0.297355,0.259395,0.956159,0.024551,0.149513,...,0.037798,0.923837,0.760233,0.931473,0.832267,0.965888,0.344429,0.052057,0.945962,0.908904
59,0.867401,0.797268,0.064659,0.058768,0.032740,0.500739,NaN,0.945554,0.027649,0.094192,...,NaN,NaN,0.574497,0.929329,0.868870,0.983064,0.209679,0.048327,0.962515,0.792598
60,0.870313,NaN,0.276795,0.013228,0.607436,0.255697,0.184245,0.949813,0.011927,0.204092,...,0.025456,0.935239,0.739701,0.828621,0.641698,0.988885,0.210891,0.030158,0.969907,0.695850
61,0.878543,NaN,0.086253,0.035579,0.689354,0.285745,NaN,0.951780,0.026027,0.089650,...,0.031333,NaN,0.749474,0.870573,0.808924,0.976856,0.097765,0.058743,NaN,0.774355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.848019,0.714955,0.345282,0.044060,0.217289,0.286300,0.213746,0.933303,0.019338,0.163362,...,0.040914,0.936746,0.703963,0.932591,0.794732,0.973849,0.443481,0.042136,0.945318,0.949573
596,0.842057,0.844408,0.167503,0.030814,0.045414,0.271550,0.299056,0.955565,0.011885,0.289069,...,0.015548,0.949698,0.737791,0.930943,0.696165,0.968152,0.431557,0.055152,0.935072,0.805257
597,0.875423,0.607010,0.052774,0.022742,0.015006,0.178868,0.350928,0.967761,0.011369,0.070731,...,0.038441,0.555965,0.206392,NaN,0.777240,0.976052,0.203283,0.056329,0.972555,0.937175
598,0.843760,0.799665,0.430911,0.028418,0.322526,0.501661,0.274369,0.961182,0.014764,0.260118,...,0.026885,0.926543,0.876002,0.910892,0.814292,0.974510,0.522833,0.066074,0.957475,0.943723


In [13]:
all_beta_normalized_tumor = all_beta_normalized_tumor.subtract(
    train_normal_avg, axis=1
)

In [14]:
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [16]:
train_tumor_mean = all_beta_normalized_tumor.mean(skipna=True, axis=0)

In [17]:
delta_beta = pd.merge(
    train_df.iloc[:-1, :1],
    pd.DataFrame(train_tumor_mean, columns=["dbeta"]),
    left_index=True,
    right_index=True,
)
update_nested_toml("preprocess.dbeta", "delta_beta_avg", delta_beta.shape[0])

In [18]:
# print(delta_beta[pd.isna(delta_beta["dbeta"])])
# record the list of feature with dbeta being NaN
update_nested_toml(
    "preprocess.dbeta",
    "NaN_dbeta_feature",
    delta_beta.loc[pd.isna(delta_beta["dbeta"]), "Unnamed: 0"].tolist(),
)
delta_beta.dropna(inplace=True, axis=0)
update_nested_toml("preprocess.dbeta", "delta_beta_avg_remove_NaN", delta_beta.shape[0])

In [19]:
dmp = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/DMP_result_0.csv")
dmp = dmp[["Unnamed: 0", "gene", "feature"]]
update_nested_toml("preprocess.dbeta", "dmp_before_dropna_shape_feature", dmp.shape[0])
dmp.dropna(inplace=True)
update_nested_toml("preprocess.dbeta", "dmp_after_dropna_shape_feature", dmp.shape[0])

In [20]:
result = pd.merge(delta_beta, dmp, on="Unnamed: 0", how="left")
update_nested_toml(
    "preprocess.dbeta", "delta_beta_avg_remove_NaN_with_gene_name", result.shape[0]
)

In [21]:
def find_max_dBeta_grouped(group):
    idx_max = group["dbeta"].abs().idxmax()
    return group.loc[idx_max]


dbeta = result.groupby("gene", as_index=False).apply(
    find_max_dBeta_grouped, include_groups=False
)

In [22]:
dbeta.columns = ["gene", "ID", "dbeta", "feature"]
dbeta = dbeta[["ID", "gene", "dbeta", "feature"]]
# DEBUG
dbeta
# END

,ID,gene,dbeta,feature
0,cg10734734,A1BG,-0.295563,Body
1,cg07849944,A2BP1,0.392910,5'UTR
2,cg14888445,A2LD1,-0.081410,Body
3,cg08300930,A2M,0.034428,Body
4,cg15384867,A2ML1,0.000501,Body
...,...,...,...,...
14414,cg24466241,ZYG11A,0.213410,Body
14415,cg24675658,ZYG11B,0.170845,TSS200
14416,cg05102190,ZYX,0.052576,TSS200
14417,cg18943946,ZZEF1,0.288934,Body


In [ ]:
# comorbidity = pd.read_csv(
#     "../external_result/matchgene174_single_3Y10__OR2.txt", sep="\t", header=None
# )
# dbeta = dbeta[
#     dbeta["gene"].isin(comorbidity[0])
# ]

# result_max_per_gene_single

In [23]:
dbeta["dbeta"] = dbeta["dbeta"].apply(lambda x: round(x, 6))
dbeta.to_csv(f"{trainOutPath}/dbeta.csv", index=False)

### Sec. 2 Filter Genes by Average Delta Beta Values
1. filter genes by dbeta values
3. filter genes by TSS position
4. plot distribution of dbeta values
5. plot PCA for normal and tumor


In [ ]:
# dbeta = pd.read_csv(f"{trainOutPath}/dbeta.csv")

#### 2.1 Filtering TSS

In [24]:
TSS = dbeta[dbeta["feature"].str.contains("TSS")]

In [25]:
TSS.to_csv(f"{trainOutPath}/dbeta_TSS.csv", index=False)

#### 2.2 Thresholding

In [26]:
threshold = 1
TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
while True:
    TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
    count = TSS_threshold.shape[0]
    if (
        config["preprocess"]["filtering"]["hyper"]["avg_dbeta_lower_bound"]
        <= count
        <= config["preprocess"]["filtering"]["hyper"]["avg_dbeta_upper_bound"]
    ):
        break
    threshold -= 0.01
threshold = round(threshold, 2)
update_nested_toml("preprocess.filtering", "threshold", threshold)

In [27]:
TSS_threshold.to_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv", index=False)

#### 2.3 Visualization

In [ ]:
# DEBUG
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(TSS_threshold["dbeta"])
plt.xlabel("delta Beta value")
plt.title("Density plot of delta Beta value")
# save the plot
plt.savefig(f"{trainOutPath}/dbeta_TSS_{threshold}.png")
plt.close()
# END

In [ ]:
# train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [29]:
normal_count = (train_df.iloc[-1, 1:] == 0).sum()
df_gene = train_df.iloc[:-1, :]
df_gene = df_gene[df_gene[df_gene.columns[0]].isin(dbeta["ID"])]
X = df_gene.iloc[:, 1:].reset_index(drop=True).T
y = [0 if i < normal_count else 1 for i in range(X.shape[0])]
# DEBUG
print(f"X shape: {X.shape}")
print(f"y shape: {len(y)}")
# END

X shape: (600, 14419)
y shape: 600


In [30]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

df = pd.DataFrame(
    {
        "Principal Component 1": X_pca[:, 0],
        "Principal Component 2": X_pca[:, 1],
        "Principal Component 3": X_pca[:, 2],
        "Class": y,
    }
)
print(df.shape)
fig = px.scatter_3d(
    df,
    x="Principal Component 1",
    y="Principal Component 2",
    z="Principal Component 3",
    color="Class",
    title="PCA of Dataset",
    color_continuous_scale="Viridis",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3",
    )
)

# fig.show()

fig.write_html(f"{trainOutPath}/preprocess_filtering_pca.html")

# open in browser
# save the plot

(600, 4)


### Sec. 3 Machine Learning

#### 3.1 Preparation

In [31]:
config = load_config(CONFIG_PATH)
threshold = config["preprocess"]["filtering"]["threshold"]
TSS_threshold = pd.read_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv")

In [ ]:
# TSS_threshold_hyper = TSS_threshold[TSS_threshold["dbeta"] > 0]
# # DEBUG
# TSS_threshold_hyper
# # END

In [33]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")
test_df = pd.read_csv(f"{testOutPath}/all_beta_normalized_1.csv")

In [34]:
X_train = train_df[train_df["Unnamed: 0"].isin(TSS_threshold["ID"])]
X_test = test_df[test_df["Unnamed: 0"].isin(TSS_threshold["ID"])]

In [35]:
X_train = X_train.iloc[:, 1:].T.values.tolist()
X_test = X_test.iloc[:, 1:].T.values.tolist()

In [36]:
config = load_config(CONFIG_PATH)
normal_count_train = config["preprocess"]["merge_and_split"][
    "training_class_distribution"
][0]

normal_count_test = config["preprocess"]["merge_and_split"][
    "testing_class_distribution"
][0]
y_train = [0 if i < normal_count_train else 1 for i in range(len(X_train))]
y_test = [0 if i < normal_count_test else 1 for i in range(len(X_test))]

#### 3.2 Training

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from utils.train_helper import set_parameters, append_to_file, record

In [38]:
selection_models = {
    "XGBoost": XGBClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(kernel="linear", random_state=42),
}

param_xgb = {
    "learning_rate": [0.001, 0.01, 0.1],
    "n_estimators": [50, 100, 200],
    "max_depth": [2, 3, 4],
    "subsample": [0.6, 0.7, 0.8],
    "colsample_bytree": [0.6, 0.7, 0.8],
    "min_child_weight": [40, 50, 60],
}

param_rf = {
    "n_estimators": [50, 100, 150, 200],
    "min_samples_split": [80, 90],
    "min_samples_leaf": [80, 90],
    "max_features": [5, 10, 20],
}

param_svm = {"kernel": ["rbf", "poly", "sigmoid"], "C": [50, 100, 200]}

param_dt = {
    "min_samples_split": [70, 80, 90],
    "min_samples_leaf": [70, 80, 90],
    "max_features": [5, 10, 20],
}

xgb_model = set_parameters(XGBClassifier(random_state=42), param_xgb)
rf_model = set_parameters(RandomForestClassifier(random_state=42), param_rf)
svm_model = set_parameters(SVC(random_state=42), param_svm)
dt_model = set_parameters(DecisionTreeClassifier(random_state=42), param_dt)

train_models = {
    "XGBoost": xgb_model,
    "Random Forest": rf_model,
    "Decision Tree": dt_model,
    "SVM": svm_model,
}


In [ ]:
for selection_model_name, selection_model in selection_models.items():
    for feature_count in range(20, 81, 5):
        rfe = RFE(estimator=selection_model, n_features_to_select=feature_count)
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_test_rfe = rfe.transform(X_test)
        selected_feature_names = (
            pd.DataFrame(TSS_threshold.iloc[rfe.support_, 1])
            .reset_index(drop=True)
            .T
        )

        label = f"{selection_model_name}_{feature_count}"

        selected_feature_names.insert(0, "selection_model_name", label)

        append_to_file(
            f"{trainOutPath}/selected_feature_names.csv", selected_feature_names
        )

        for train_model_name, train_model in train_models.items():
            train_model.fit(X_train_rfe, y_train)
            best_params = train_model.best_params_

            best_score = train_model.best_score_

            record(
                train_model.best_estimator_,
                X_train_rfe,
                y_train,
                selection_model_name,
                train_model_name,
                feature_count,
                best_score,
                best_params,
                f"{trainOutPath}",
            )
            record(
                train_model.best_estimator_,
                X_test_rfe,
                y_test,
                selection_model_name,
                train_model_name,
                feature_count,
                best_score,
                best_params,
                f"{testOutPath}",
            )

#### 3.3 Visualization

In [6]:
fpr_tpr_train = pd.read_csv(f"{trainOutPath}/fpr_tpr.csv")
fpr_tpr_test = pd.read_csv(f"{testOutPath}/fpr_tpr.csv")
rfe_train = pd.read_csv(f"{trainOutPath}/rfe.csv")
rfe_test = pd.read_csv(f"{testOutPath}/rfe.csv")

In [7]:
rfe_j = pd.merge(rfe_train, rfe_test, on=["selection_model", "train_model", "features"], suffixes=('_train', '_test'))
fpr_tpr_j = pd.merge(fpr_tpr_train, fpr_tpr_test, on=["selection_model", "train_model", "features"], suffixes=('_train', '_test'))
J = pd.merge(rfe_j, fpr_tpr_j, on=["selection_model", "train_model", "features", "AUC_train", "AUC_test"])

In [8]:
import ast

J["fpr_train"] = J["fpr_train"].apply(ast.literal_eval)
J["tpr_train"] = J["tpr_train"].apply(ast.literal_eval)
J["fpr_test"] = J["fpr_test"].apply(ast.literal_eval)
J["tpr_test"] = J["tpr_test"].apply(ast.literal_eval)

In [9]:
from utils.painter import plot_roc_curve, create_performance_barchart

In [10]:
J['accuracy_diff'] = J['accuracy_train'] - J['accuracy_test']
J['recall_diff'] = J['recall_train'] - J['recall_test']
J['f1_score_diff'] = J['f1_score_train'] - J['f1_score_test']
J['AUC_diff'] = J['AUC_train'] - J['AUC_test']
J['MCC_diff'] = J['MCC_train'] - J['MCC_test']

In [11]:
# tweakable width and height
plot_roc_curve(J, "ROC Curves on Training Set", f"{trainOutPath}/roc_train.html")

ROC curve saved to ../lung/result/GDC_lung_tissue/train80/roc_train.html


In [12]:
# tweakable width and height
plot_roc_curve(J, "ROC Curves on Testing Set", f"{testOutPath}/roc_test.html", mode="test")

ROC curve saved to ../lung/result/GDC_lung_tissue/test20/roc_test.html


#### 3.4 Filter Combinations of "selection_model", "training_model", and "features"

In [13]:
# eliminate the those with abs accuracy difference greater than 0.1
J = J[abs(J["accuracy_diff"]) < 0.1]

In [14]:
# plot difference
performance_metrics = ['accuracy_diff', 'recall_diff', 'f1_score_diff', 'AUC_diff', 'MCC_diff']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(f"{trainOutPath}/performance_diff_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_diff": "blue",
    "recall_diff": "red",
    "f1_score_diff": "green",
    "AUC_diff": "purple",
    "MCC_diff": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{trainOutPath}/performance_diff_grouped_by_train_model.html",
    title="Grouped Performance Difference between Training and Testing Set",
    x_axis_label="Performance Difference (Training - Testing)",
    y_axis_label="Train Model",
    orientation="h",
)

Performance difference saved to ../lung/result/GDC_lung_tissue/train80/performance_diff_grouped_by_train_model.html


In [15]:
J = J[["selection_model", "train_model", "features", "accuracy_test", "recall_test", "f1_score_test", "AUC_test", "MCC_test"]]

In [16]:
# group by train_model, for each train_model, calculate the mean of each performance metric
performance_metrics = ['accuracy_test', 'recall_test',
                       'f1_score_test', 'AUC_test', 'MCC_test']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(
    f"{trainOutPath}/performance_metrics_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_test": "blue",
    "recall_test": "red",
    "f1_score_test": "green",
    "AUC_test": "purple",
    "MCC_test": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{trainOutPath}/performance_metrics_grouped_by_train_model.html",
    title="Grouped Performance Metrics by Train Model",
    x_axis_label="Performance",
    y_axis_label="Train Model",
    orientation="h",
)
best_train_model = ground_by_train_model['MCC_test'].idxmax()
print(f"Best train model: {best_train_model}")
ground_by_feature = J[J['train_model'] == best_train_model].groupby('features')[
    performance_metrics].mean()
ground_by_feature['features'] = ground_by_feature.index
ground_by_feature.to_csv(
    f"{trainOutPath}/performance_metrics_grouped_by_feature.csv", index=False)
create_performance_barchart(
    df=ground_by_feature,
    color_mapping=color_mapping,
    metric="features",
    out_path=f"{trainOutPath}/performance_metrics_grouped_by_feature.html",
    title="Grouped Performance Metrics by Feature",
    x_axis_label="Performance",
    y_axis_label="Feature",
    orientation="h",
)
best_num_of_feature = ground_by_feature['MCC_test'].idxmax()
print(f"Best number of feature: {best_num_of_feature}")
best_performance_records = J[(J['train_model'] == best_train_model) & (
    J['features'] == best_num_of_feature)]
best_performance_records.to_csv(
    f"{trainOutPath}/best_performance_records.csv", index=False)

Performance difference saved to ../lung/result/GDC_lung_tissue/train80/performance_metrics_grouped_by_train_model.html
Best train model: SVM
Performance difference saved to ../lung/result/GDC_lung_tissue/train80/performance_metrics_grouped_by_feature.html
Best number of feature: 70


In [26]:
first = True
gene_set = set()
with open(f"{trainOutPath}/selected_feature_names.csv", "r") as f:
    for line in f:
        if first:
            first = False
            continue
        if line.split(",")[0].endswith(str(best_num_of_feature)):
            selected_feature_names = line.split(",")[1:]
            selected_feature_names[-1] = selected_feature_names[-1].strip()
            gene_set.update(selected_feature_names)

gene_list = pd.DataFrame(list(gene_set), columns=["gene"])
gene_list.to_csv(f"{trainOutPath}/selected_feature_set.csv", index=False)

### Sec. 4 Clustering

#### 4.1 load data

In [6]:
import pandas as pd
import numpy as np
from utils.clustering_helper import hierarchical_clustering, check_distance_matrix

In [7]:
dbeta_path = config["clustering"]["hyper"]["dbeta_file"]
gene_set_file = config["clustering"]["hyper"]["gene_set_file"]
bp_file = config["clustering"]["hyper"]["bp_file"]
cc_file = config["clustering"]["hyper"]["cc_file"]
mf_file = config["clustering"]["hyper"]["mf_file"]
terms_count_file = config["clustering"]["hyper"]["terms_count_file"]

In [8]:
dbeta = pd.read_csv(f"{trainOutPath}/{dbeta_path}.csv", index_col=0)
gene_set = pd.read_csv(f"{trainOutPath}/{gene_set_file}.csv", index_col=0)
distance_matrix_bp = pd.read_csv(f"{trainOutPath}/{bp_file}.csv", index_col=0)
distance_matrix_cc = pd.read_csv(f"{trainOutPath}/{cc_file}.csv", index_col=0)
distance_matrix_mf = pd.read_csv(f"{trainOutPath}/{mf_file}.csv", index_col=0)
terms_count = pd.read_csv(f"{trainOutPath}/{terms_count_file}.csv", index_col=0)

In [9]:
# replace NaN with 0
distance_matrix_bp = distance_matrix_bp.fillna(0)
distance_matrix_cc = distance_matrix_cc.fillna(0)
distance_matrix_mf = distance_matrix_mf.fillna(0)

In [10]:
# reindex distance matrix
index_bp = distance_matrix_bp.index
index_cc = distance_matrix_cc.index
index_mf = distance_matrix_mf.index
index = index_bp.union(index_cc).union(index_mf)
distance_matrix_bp_ = distance_matrix_bp.reindex(index=index, columns=index, fill_value=0)
distance_matrix_cc_ = distance_matrix_cc.reindex(index=index, columns=index, fill_value=0)
distance_matrix_mf_ = distance_matrix_mf.reindex(index=index, columns=index, fill_value=0)

In [11]:
# make a array of distance matrix for each ontology
distance_matrix = []

distance_matrix.append(distance_matrix_bp_)
distance_matrix.append(distance_matrix_cc_)
distance_matrix.append(distance_matrix_mf_)

#### 4.2 Weighted Sum

In [12]:
weight = [count for count in terms_count["count"]]
weight = weight / np.sum(weight)
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])

valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks

weight_sums = valid_weights.sum(axis=0)

normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)


weighted_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)

weighted_sum_dataframe.head()

,ABCA4,ACCN4,ADORA2B,AGAP2,ALDH1A3,ALDH1L1,AP4B1,APBB1,APOC3,ARMC3,...,WDR81,ZBTB25,ZBTB34,ZFAND3,ZMYM2,ZNF131,ZNF273,ZNF799,ZNF836,ZNRD1
ABCA4,0.000000,0.716153,0.766936,0.664578,0.723304,0.815421,0.696451,0.735206,0.682787,0.627098,...,0.762085,0.805304,0.829727,0.208969,0.773070,0.847298,0.826285,0.790825,0.843022,0.820289
ACCN4,0.716153,0.000000,0.748446,0.786401,0.839745,0.885003,0.704340,0.713146,0.803960,0.676526,...,0.808522,0.833000,0.830184,0.149010,0.754840,0.887083,0.869420,0.917234,0.922587,0.844681
ADORA2B,0.766936,0.748446,0.000000,0.771497,0.815498,0.865735,0.721395,0.580100,0.676604,0.676832,...,0.775264,0.676483,0.676672,0.131554,0.723006,0.734275,0.795312,0.780221,0.854803,0.816780
AGAP2,0.664578,0.786401,0.771497,0.000000,0.771143,0.820128,0.579847,0.642304,0.683043,0.586894,...,0.601978,0.693000,0.738650,0.150528,0.688880,0.772181,0.718949,0.690647,0.758241,0.771553
ALDH1A3,0.723304,0.839745,0.815498,0.771143,0.000000,0.665803,0.780580,0.685921,0.766871,0.566026,...,0.799398,0.817203,0.815747,0.174815,0.720761,0.846424,0.850098,0.820727,0.880350,0.811062


In [13]:
cluster_result_weighted = hierarchical_clustering(
    weighted_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{trainOutPath}/hierarchical_clustering_weighted_sum.png",
)

chosen number of clusters: 3


In [14]:
cluster_result_weighted.head()

,gene,cluster
0,ABCA4,1
1,ACCN4,1
2,ADORA2B,1
3,AGAP2,1
4,ALDH1A3,1


#### 4.3 Simple average

In [15]:
weight = [1, 1, 1]
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])
valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks
weight_sums = valid_weights.sum(axis=0)
normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)
simple_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)
simple_sum_dataframe.head()

,ABCA4,ACCN4,ADORA2B,AGAP2,ALDH1A3,ALDH1L1,AP4B1,APBB1,APOC3,ARMC3,...,WDR81,ZBTB25,ZBTB34,ZFAND3,ZMYM2,ZNF131,ZNF273,ZNF799,ZNF836,ZNRD1
ABCA4,0.000000,0.525000,0.720667,0.553333,0.697667,0.780000,0.672667,0.631667,0.676000,0.496000,...,0.689667,0.680667,0.765333,0.275333,0.705000,0.792333,0.629667,0.621333,0.650667,0.764000
ACCN4,0.525000,0.000000,0.512333,0.637000,0.775333,0.837333,0.657667,0.462667,0.710000,0.575667,...,0.738667,0.781667,0.769000,0.196333,0.696333,0.847667,0.806333,0.869333,0.875333,0.782333
ADORA2B,0.720667,0.512333,0.000000,0.712333,0.758000,0.824667,0.657333,0.514667,0.657000,0.565000,...,0.720333,0.683000,0.680333,0.173333,0.668667,0.762667,0.744333,0.779667,0.821667,0.759333
AGAP2,0.553333,0.637000,0.712333,0.000000,0.615333,0.700667,0.541333,0.523333,0.590333,0.300000,...,0.514667,0.477667,0.636333,0.198333,0.547000,0.690333,0.487667,0.500333,0.537333,0.657667
ALDH1A3,0.697667,0.775333,0.758000,0.615333,0.000000,0.455333,0.660000,0.626667,0.676333,0.289333,...,0.678000,0.748333,0.741000,0.230333,0.611333,0.789333,0.753667,0.759667,0.793000,0.752333


In [16]:
cluster_result_simple = hierarchical_clustering(
    simple_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{trainOutPath}/hierarchical_clustering_simple_sum.png",
)

chosen number of clusters: 3


In [17]:
cluster_result_simple.head()

,gene,cluster
0,ABCA4,2
1,ACCN4,2
2,ADORA2B,2
3,AGAP2,2
4,ALDH1A3,3


#### 4.4 Consensus clustering 

In [18]:
cluster_bp = hierarchical_clustering(
    distance_matrix_bp, out_path=f"{trainOutPath}/hierarchical_clustering_bp.png"
)
cluster_cc = hierarchical_clustering(
    distance_matrix_cc, out_path=f"{trainOutPath}/hierarchical_clustering_cc.png"
)
cluster_mf = hierarchical_clustering(
    distance_matrix_mf, out_path=f"{trainOutPath}/hierarchical_clustering_mf.png"
)

Best number of clusters: 23
Best number of clusters: 2
Best number of clusters: 4


In [19]:
cluster_bp.columns = ["gene", "cluster_bp"]
cluster_cc.columns = ["gene", "cluster_cc"]
cluster_mf.columns = ["gene", "cluster_mf"]
cluster_bp_cc = pd.merge(cluster_bp, cluster_cc, on="gene", how="outer")
cluster_go = pd.merge(cluster_bp_cc, cluster_mf, on="gene", how="outer")
cluster_go = cluster_go.fillna(-1)
print(cluster_go.shape)
cluster_go.head()

(294, 4)


,gene,cluster_bp,cluster_cc,cluster_mf
0,ABCA4,23.0,1.0,2.0
1,ACCN4,23.0,1.0,2.0
2,ADORA2B,12.0,1.0,2.0
3,AGAP2,22.0,2.0,2.0
4,ALDH1A3,9.0,2.0,1.0


In [20]:
num_genes = cluster_go.shape[0]
consensus_matrix = np.zeros((num_genes, num_genes))

for i in range(num_genes):
    for j in range(i, num_genes):
        if cluster_go.iloc[i]["cluster_bp"] == cluster_go.iloc[j]["cluster_bp"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_cc"] == cluster_go.iloc[j]["cluster_cc"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_mf"] == cluster_go.iloc[j]["cluster_mf"]:
            consensus_matrix[i][j] += 1


consensus_matrix = pd.DataFrame(
    consensus_matrix, index=cluster_go["gene"], columns=cluster_go["gene"]
)
consensus_matrix += consensus_matrix.T


distance_matrix_consensus = 1 - consensus_matrix / 3
np.fill_diagonal(distance_matrix_consensus.values, 0)


distance_matrix_consensus.head()

gene,ABCA4,ACCN4,ADORA2B,AGAP2,ALDH1A3,ALDH1L1,AP4B1,APBB1,APOC3,ARMC3,...,WDR81,ZBTB25,ZBTB34,ZFAND3,ZMYM2,ZNF131,ZNF273,ZNF799,ZNF836,ZNRD1
gene,,,,,,,,,,,,,,,,,,,,,
ABCA4,0.000000,0.000000,0.333333,0.666667,1.000000,1.000000,0.666667,0.666667,0.666667,1.000000,...,0.666667,1.000000,1.000000,0.666667,0.666667,1.000000,1.000000,1.000000,1.000000,0.666667
ACCN4,0.000000,0.000000,0.333333,0.666667,1.000000,1.000000,0.666667,0.666667,0.666667,1.000000,...,0.666667,1.000000,1.000000,0.666667,0.666667,1.000000,1.000000,1.000000,1.000000,0.666667
ADORA2B,0.333333,0.333333,0.000000,0.666667,1.000000,1.000000,0.666667,0.666667,0.666667,1.000000,...,0.666667,1.000000,1.000000,0.666667,0.666667,1.000000,1.000000,1.000000,1.000000,0.666667
AGAP2,0.666667,0.666667,0.666667,0.000000,0.666667,0.666667,0.000000,0.333333,0.333333,0.666667,...,0.333333,0.666667,0.666667,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.333333
ALDH1A3,1.000000,1.000000,1.000000,0.666667,0.000000,0.333333,0.666667,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,1.000000,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667


In [21]:
cluster_result_consensus = hierarchical_clustering(
    distance_matrix_consensus,
    range_min=2,
    range_max=4,
    cluster_number=4,
    out_path=f"{trainOutPath}/hierarchical_clustering_consensus.png",
)

chosen number of clusters: 4


In [22]:
cluster_result_consensus.head()

,gene,cluster
0,ABCA4,1
1,ACCN4,1
2,ADORA2B,1
3,AGAP2,2
4,ALDH1A3,3


#### 4.5 Compare 

In [23]:
from utils.clustering_helper import hierarchical_clustering_compare

hierarchical_clustering_compare(
    [weighted_sum_dataframe, simple_sum_dataframe, distance_matrix_consensus],
    ["Weighted Average", "Simple Average", "Consensus"],
    out_path=f"{trainOutPath}/hierarchical_clustering_compare.png",
)

Best number of clusters for Weighted Average: 2
Best number of clusters for Simple Average: 2
Best number of clusters for Consensus: 5


In [24]:
dbeta["ID"] = dbeta.index

In [25]:
# column gene isin weighted_sum_dataframe
weighted_dbeta = dbeta[dbeta["gene"].isin(weighted_sum_dataframe.index)]
simple_dbeta = dbeta[dbeta["gene"].isin(simple_sum_dataframe.index)]
consensus_dbeta = dbeta[dbeta["gene"].isin(distance_matrix_consensus.index)]

In [26]:
weighted_dbeta.merge(cluster_result_weighted, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_weighted.csv", index=False
)
simple_dbeta.merge(cluster_result_simple, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_simple.csv", index=False
)
consensus_dbeta.merge(cluster_result_consensus, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_consensus.csv", index=False
)

### Sec. 5 Combination

#### 5.1 preparation

In [6]:
import pandas as pd

In [7]:
dbeta_path = config["combination"]["hyper"]["dbeta_file"]
dbeta = pd.read_csv(f"{trainOutPath}/{dbeta_path}.csv")
dbeta

,gene,dbeta,feature,ID,cluster
0,ABCA4,0.443312,TSS1500,cg12869058,1
1,ACCN4,0.380599,TSS1500,cg24586121,1
2,ADORA2B,0.385252,TSS200,cg05916671,1
3,AGAP2,0.465194,TSS1500,cg22505898,2
4,ALDH1A3,0.391162,TSS1500,cg17797372,3
...,...,...,...,...,...
289,ZNF131,0.389198,TSS1500,cg22042060,4
290,ZNF273,0.314191,TSS1500,cg15890852,4
291,ZNF799,0.411895,TSS200,cg27037109,4
292,ZNF836,0.357159,TSS1500,cg08672999,4


In [8]:
gene_list = pd.read_csv(f"{trainOutPath}/selected_feature_set.csv")
gene_list

,gene
0,CYB5B
1,TMEM214
2,RIPPLY2
3,SLFN11
4,PUF60
...,...
156,PRKCDBP
157,EDC4
158,TPPP3
159,KCNG3


In [9]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")
test_df = pd.read_csv(f"{testOutPath}/all_beta_normalized_1.csv")

In [10]:
# param_grids = {
#     "XGBoost": {
#         "learning_rate": [0.001, 0.01, 0.1],
#         "n_estimators": [50, 100, 200],
#         "max_depth": [2, 3, 4],
#         "subsample": [0.6, 0.7, 0.8],
#         "colsample_bytree": [0.6, 0.7, 0.8],
#         "min_child_weight": [40, 50, 60],
#     },
#     "Random_Forest": {
#         "n_estimators": [50, 100, 150, 200],
#         "min_samples_split": [80, 90],
#         "min_samples_leaf": [80, 90],
#         "max_features": [5, 10, 20],
#     },
#     "SVM": {"kernel": ["rbf", "poly", "sigmoid"], "C": [50, 100, 200]},
#     "Decision_Tree": {
#         "min_samples_split": [70, 80, 90],
#         "min_samples_leaf": [70, 80, 90],
#         "max_features": [5, 10, 20],
#     },
# }

In [11]:
# X_train = train_df[train_df['Unnamed: 0'].isin(dbeta['ID'])]
# X_test = test_df[test_df['Unnamed: 0'].isin(dbeta['ID'])]


In [12]:
# from itertools import product
# from xgboost import XGBClassifier

# from sklearn.ensemble import RandomForestClassifier

# from sklearn.svm import SVC

# from sklearn.tree import DecisionTreeClassifier

# groups = dbeta['cluster'].unique()
# if len(groups) < 3:
#     raise ValueError(
#         "Not all four groups are present in the DataFrame.")

# genes_by_group = {
#     group: dbeta[dbeta['cluster'] == group]['gene'].tolist() for group in groups}

# combs = [
#     combination for combination in product(*genes_by_group.values())
#     if len(set(combination)) == 4  # Ensure genes are distinct
# ]
# print(f"Number of combinations: {len(combs)}")
# models = {
#     "XGBoost": XGBClassifier(random_state=42),
#     "Random_Forest": RandomForestClassifier(random_state=42),
#     "SVM": SVC(random_state=42),
#     "Decision_Tree": DecisionTreeClassifier(random_state=42),
# }

In [13]:
# import os
# import pandas as pd

# from sklearn.model_selection import GridSearchCV

# from sklearn.metrics import (
#     confusion_matrix,
#     precision_score,
#     accuracy_score,
#     matthews_corrcoef,
#     f1_score,
#     recall_score,
#     roc_curve,
#     auc,

# )


# def get_ID(genes: tuple) -> tuple:
#     return tuple(dbeta[dbeta['gene'].isin(genes)]['ID'])


# def prepare_data(df: pd.DataFrame, genes: tuple = None) -> list:
#     return df[df['Unnamed: 0'].isin(list(get_ID(genes)))].iloc[:, 1:].T.values.tolist()


# def set_parameters(model: object, param_grid: dict) -> object:
#     gs = GridSearchCV(
#         estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2
#     )
#     return gs


# def append_to_file(file_name: str, data: pd.DataFrame) -> None:
#     if not os.path.isfile(file_name):
#         data.to_csv(file_name, index=False)
#     else:
#         data.to_csv(file_name, index=False, mode="a", header=False)


# def record(
#     y_pred_on_X,
#     y,
#     model_name: str,
#     best_score: float,
#     best_params: dict,
#     filepath: str,
#     filename: str,
#     combination: tuple
# ) -> None:

#     tn, fp, _, _ = confusion_matrix(y, y_pred_on_X).ravel()
#     accuracy = accuracy_score(y, y_pred_on_X)
#     recall = recall_score(y, y_pred_on_X)
#     if (tn + fp) == 0:
#         print("Warning: No true negative found. Setting specificity to 0.")
#         specificity = 0
#     else:
#         specificity = tn / (tn + fp)
#     precision = precision_score(y, y_pred_on_X)
#     fpr, tpr, _ = roc_curve(y, y_pred_on_X)
#     roc_auc = auc(fpr, tpr)
#     fpr = fpr.tolist()
#     tpr = tpr.tolist()

#     append_to_file(
#         f"{filepath}/{filename}.csv",
#         pd.DataFrame(
#             [
#                 {
#                     "model": model_name,
#                     "accuracy": accuracy,
#                     "recall": recall,
#                     "specificity": specificity,
#                     "precision": precision,
#                     "f1_score": f1_score(y, y_pred_on_X),
#                     "AUC": roc_auc,
#                     "MCC": matthews_corrcoef(y, y_pred_on_X),
#                     "Mean cross-validated score": best_score,
#                     "Best parameters": best_params,
#                     **{f"ID{i+1}": gene for i, gene in enumerate(combination) if gene is not None}
#                 }
#             ]
#         ),
#     )
#     append_to_file(
#         f"{filepath}/{filename}_fpr_tpr.csv",
#         pd.DataFrame(
#             [
#                 {
#                     "model": model_name,
#                     "fpr": fpr,
#                     "tpr": tpr,
#                     "AUC": roc_auc,
#                 }
#             ]
#         ),
#     )


# def run(models: dict, X_train_, y_train, X_test_, y_test, folder: str, filename: str, discard_overfitting: bool = False):
#     grids = {}
#     for model_name, model in models.items():
#         grids[model_name] = set_parameters(model, param_grids[model_name])
#     for model_name, model in grids.items():
#         for combination in combs:
#             y_train = y_train
#             y_test = y_test
#             X_train = prepare_data(X_train_, combination)
#             X_test = prepare_data(X_test_, combination)
            

#             model.fit(X_train, y_train)

#             best_params = model.best_params_

#             best_score = model.best_score_
#             y_pred_on_X_train = model.best_estimator_.predict(X_train)
#             y_pred_on_X_test = model.best_estimator_.predict(X_test)

#             if discard_overfitting:
#                 train_acc = accuracy_score(y_train, y_pred_on_X_train)
#                 test_acc = accuracy_score(y_test, y_pred_on_X_test)
#                 if abs(train_acc - test_acc) > 0.1:
#                     print("Train accuracy: ", round(train_acc, 6))
#                     print("Test accuracy: ", round(test_acc, 6))
#                     print(f"========={model_name} overfitting =========")
#                     continue
#             record(
#                 y_pred_on_X_train,
#                 y_train,
#                 model_name,
#                 best_score,
#                 best_params,
#                 folder,
#                 filename,
#                 combination
#             )
#             record(
#                 y_pred_on_X_test,
#                 y_test,
#                 model_name,
#                 best_score,
#                 best_params,
#                 folder,
#                 filename,
#                 combination
#             )

#### 5.2 training

In [14]:
# import os
# import pandas as pd

# from itertools import combinations
# from sklearn.model_selection import GridSearchCV
# from xgboost import XGBClassifier

# from sklearn.ensemble import RandomForestClassifier

# from sklearn.svm import SVC

# from sklearn.tree import DecisionTreeClassifier

# from sklearn.metrics import (
#     confusion_matrix,
#     precision_score,
#     accuracy_score,
#     matthews_corrcoef,
#     f1_score,
#     recall_score,
#     roc_curve,
#     auc,

# )
# from typing import List
# from itertools import product
# from typing import Literal


# class CombinationHelper:
#     def __init__(self,
#                  y_train: pd.Series,
#                  X_train: pd.DataFrame,
#                  y_test: pd.Series,
#                  X_test: pd.DataFrame,
#                  dbeta: pd.DataFrame,
#                  gene_list: List[str],
#                  param_grids: dict) -> None:
#         self.y_train = y_train
#         self.X_train = X_train
#         self.y_test = y_test
#         self.X_test = X_test
#         self.dbeta = dbeta
#         self.gene_list = gene_list
#         self.param_grids = param_grids

#         self.dbeta = self.dbeta[self.dbeta["gene"].isin(
#             self.gene_list["gene"])]
#         self.X_train = self.X_train[self.X_train['Unnamed: 0'].isin(
#             self.dbeta['ID'])]
#         self.X_test = self.X_test[self.X_test['Unnamed: 0'].isin(
#             self.dbeta['ID'])]

#         groups = dbeta['cluster'].unique()
#         if len(groups) < 3:
#             raise ValueError(
#                 "Not all four groups are present in the DataFrame.")

#         genes_by_group = {
#             group: dbeta[dbeta['cluster'] == group]['gene'].tolist() for group in groups}

#         self.combs = [
#             combination for combination in product(*genes_by_group.values())
#             if len(set(combination)) == 4  # Ensure genes are distinct
#         ]

#         self.models = {
#             "XGBoost": XGBClassifier(),
#             "Random_Forest": RandomForestClassifier(),
#             "SVM": SVC(),
#             "Decision_Tree": DecisionTreeClassifier(),
#         }
#     def get_ID(self, genes: tuple) -> tuple:
#         return tuple(self.dbeta[self.dbeta['gene'].isin(genes)]['ID'])
#     def prepare_data(self, type: Literal["train", "test"] = "train", genes: tuple = None) -> list:
#         if type == "train":
#             return self.X_train[self.X_train['Unnamed: 0'].isin(self.get_ID(genes))].iloc[:, 1:].T.values.tolist()
#         elif type == "test":
#             return self.X_test[self.X_test['Unnamed: 0'].isin(self.get_ID(genes))].iloc[:, 1:].T.values.tolist()
#         else:
#             raise ValueError("Invalid type. Must be 'train' or 'test'.")

#     def set_parameters(self, model: object, model_name: str) -> object:
#         print(f"Setting parameters for {model_name}")
#         print(f"Param grid: {self.param_grids[model_name]}")
#         gs = GridSearchCV(
#             estimator=model, param_grid=self.param_grids[model_name], cv=5, n_jobs=-1, verbose=2
#         )
#         return gs

#     def append_to_file(file_name: str, data: pd.DataFrame) -> None:
#         if not os.path.isfile(file_name):
#             data.to_csv(file_name, index=False)
#         else:
#             data.to_csv(file_name, index=False, mode="a", header=False)

#     def record(
#         self,
#         y_pred_on_X: pd.Series,
#         y: pd.Series,
#         model_name: str,
#         best_score: float,
#         best_params: dict,
#         filepath: str,
#         filename: str,
#         combination: tuple
#     ) -> None:

#         tn, fp, _, _ = confusion_matrix(y, y_pred_on_X).ravel()
#         accuracy = accuracy_score(y, y_pred_on_X)
#         recall = recall_score(y, y_pred_on_X)
#         if (tn + fp) == 0:
#             print("Warning: No true negative found. Setting specificity to 0.")
#             specificity = 0
#         else:
#             specificity = tn / (tn + fp)
#         precision = precision_score(y, y_pred_on_X)
#         fpr, tpr, _ = roc_curve(y, y_pred_on_X)
#         roc_auc = auc(fpr, tpr)
#         fpr = fpr.tolist()
#         tpr = tpr.tolist()

#         self.append_to_file(
#             f"{filepath}/{filename}.csv",
#             pd.DataFrame(
#                 [
#                     {
#                         "model": model_name,
#                         "accuracy": accuracy,
#                         "recall": recall,
#                         "specificity": specificity,
#                         "precision": precision,
#                         "f1_score": f1_score(y, y_pred_on_X),
#                         "AUC": roc_auc,
#                         "MCC": matthews_corrcoef(y, y_pred_on_X),
#                         "Mean cross-validated score": best_score,
#                         "Best parameters": best_params,
#                         **{f"ID{i+1}": gene for i, gene in enumerate(combination) if gene is not None}
#                     }
#                 ]
#             ),
#         )
#         self.append_to_file(
#             f"{filepath}/{filename}_fpr_tpr.csv",
#             pd.DataFrame(
#                 [
#                     {
#                         "model": model_name,
#                         "fpr": fpr,
#                         "tpr": tpr,
#                         "AUC": roc_auc,
#                     }
#                 ]
#             ),
#         )

#     def train(self, folder: str, filename: str, discard_overfitting: bool = False):
#         for model_name, model in self.models.items():
#             for combination in self.combs:
#                 y_train = self.y_train
#                 y_test = self.y_test
#                 X_train = self.prepare_data("train", combination)
#                 X_test = self.prepare_data("test", combination)
#                 model = self.set_parameters(model, model_name)
                
#                 model.fit(X_train, y_train)

#                 best_params = model.best_params_

#                 best_score = model.best_score_
#                 y_pred_on_X_train = model.best_estimator_.predict(X_train)
#                 y_pred_on_X_test = model.best_estimator_.predict(X_test)

#                 if discard_overfitting:
#                     train_acc = accuracy_score(y_train, y_pred_on_X_train)
#                     test_acc = accuracy_score(y_test, y_pred_on_X_test)
#                     if abs(train_acc - test_acc) > 0.1:
#                         print("Train accuracy: ", round(train_acc, 6))
#                         print("Test accuracy: ", round(test_acc, 6))
#                         print(f"========={model_name} overfitting =========")
#                         continue
#                 self.record(
#                     y_pred_on_X_train,
#                     y_train,
#                     model_name,
#                     best_score,
#                     best_params,
#                     folder,
#                     filename,
#                     combination
#                 )
#                 self.record(
#                     y_pred_on_X_test,
#                     y_test,
#                     model_name,
#                     best_score,
#                     best_params,
#                     folder,
#                     filename,
#                     combination
#                 )


In [10]:
from utils.combination_helper import CombinationHelper

In [11]:
helper = CombinationHelper(
    train_df=train_df,
    test_df=test_df,
    dbeta=dbeta,
    gene_list=gene_list,
)

In [12]:
helper.setup_dbeta()
helper.setup_train_test()
helper.setup_combinations()
helper.setup_estimators()
helper.setup_grids()
helper.setup_grid_estimator()
helper.setup_grid_estimator()


In [ ]:
helper.train(
    train_folder=trainOutPath,
    test_folder=testOutPath,
    filename="combination",
    discard_overfitting=True
)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
